In [ ]:
!pip install cryptofeed

In [17]:
#
# This product based on cryptofeed https://github.com/bmoscon/cryptofeed software developed by Bryant Moscon (http://www.bryantmoscon.com/) Copyright (C) 2017-2022 Bryant Moscon - bmoscon@gmail.com
#

import multiprocessing
import os
import argparse
from datetime import datetime


import ipywidgets as widgets
from IPython.display import display
import threading
import time
from cryptofeed import FeedHandler
from cryptofeed.backends.backend import BackendCallback
from cryptofeed.backends.socket import SocketCallback
from cryptofeed.defines import TRADES
from cryptofeed.exchanges import OKX

QUEST_HOST = 'questdb'
QUEST_PORT = 9009

class QuestCallback(SocketCallback):
    def __init__(self, host='host.docker.internal', port=9009, **kwargs):
        super().__init__(f"tcp://{host}", port=port, **kwargs)
        self.numeric_type = float
        self.none_to = None

    async def writer(self):
        while True:
            try:
                await self.connect()
            except Exception as e:
                print("Connection error:", e)
                break
            async with self.read_queue() as update:
                update = "\n".join(update) + "\n"
                try:
                    self.conn.write(update.encode())
                except Exception as e:
                    print("Write error:", e)
                    break  

class TradeQuest(QuestCallback, BackendCallback):
    default_key = 'trades'

    async def write(self, data):
        update = f'{self.key},symbol={data["symbol"]},side={data["side"]} price={data["price"]},amount={data["amount"]} {int(data["timestamp"] * 1_000_000_000)}'
        await self.queue.put(update)
        
def run_feed():
    # In a separate process, we don't need to worry about Jupyter's event loop.
    handler = FeedHandler()
    symbols = [
        'ADA-USDT',
        'BTC-USDT',       
        'ETH-BTC',
        'ETH-USDT',
    ]
    
    handler.add_feed(OKX(channels=[TRADES], symbols=symbols,
                           callbacks={TRADES: TradeQuest(host=QUEST_HOST, port=QUEST_PORT)}))
    handler.run()




In [18]:
# Start the feed handler in a separate process.
feed_process = multiprocessing.Process(target=run_feed)
feed_process.start()

# Create a status indicator widget
status = widgets.Label("Feed is running...")
display(status)

# Start a background thread that periodically updates the status label
def update_status():
    while feed_process.is_alive():
        timestamp = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
        status.value = f"Feed is running... Last check: {timestamp}"
        time.sleep(1)
    status.value = "Feed stopped."

threading.Thread(target=update_status, daemon=True).start()


Label(value='Feed is running...')

In [19]:
feed_process.terminate()